In [ ]:
!pip install tensorflow

In [ ]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import tensorflow as tf
import tensorflow.keras.backend as K
from itertools import islice, cycle
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow import keras
from tqdm import tqdm



In [ ]:
interactions_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ИТМО/Recsys/5/interactions_processed_kion.csv')
users_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ИТМО/Recsys/5/users_processed_kion.csv')
items_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ИТМО/Recsys/5/items_processed_kion.csv')

In [ ]:
users_df.head()

In [ ]:
items_df = items_df.rename(columns = {'id' : 'item_id'})

In [ ]:
items_df.head()

In [ ]:
interactions_df.head()

In [ ]:
user_cat_feats = ["age", "income", "sex", "kids_flg"]
users_ohe_df = users_df.user_id
for feat in user_cat_feats:
    ohe_feat_df = pd.get_dummies(users_df[feat], prefix=feat)
    users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

users_ohe_df.head()


In [ ]:
items_df.head()

In [ ]:
item_cat_feats = ['content_type', 'release_year_cat',
                  'for_kids', 'age_rating',
                  'studios', 'countries', 'directors']

items_ohe_df = items_df.item_id

for feat in item_cat_feats:
    ohe_feat_df = pd.get_dummies(items_df[feat], prefix=feat)
    items_ohe_df = pd.concat([items_ohe_df, ohe_feat_df], axis=1)

items_ohe_df.head()

In [ ]:
vectorizer = TfidfVectorizer()
genres_tfidf = vectorizer.fit_transform(items_df['genres'].astype(str))
genres_tfidf_df = pd.DataFrame(genres_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
items_ohe_df = pd.concat([items_ohe_df, genres_tfidf_df], axis=1)
items_ohe_df

In [ ]:
interactions_df.item_id.value_counts()

In [ ]:
print(f"N users before: {interactions_df.user_id.nunique()}")
print(f"N items before: {interactions_df.item_id.nunique()}\n")

interactions_df = interactions_df[interactions_df.watched_pct > 10]

valid_users = []

c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
    if entries > 10:
        valid_users.append(user_id)

valid_items = []

c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
    if entries > 10:
        valid_items.append(item_id)

interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

print(f"N users after: {interactions_df.user_id.nunique()}")
print(f"N items after: {interactions_df.item_id.nunique()}")

In [ ]:
common_users = set(interactions_df.user_id.unique()).intersection(set(users_ohe_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_ohe_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df.user_id.isin(common_users)]

In [ ]:
interactions_df["uid"] = interactions_df["user_id"].astype("category")
interactions_df["uid"] = interactions_df["uid"].cat.codes

interactions_df["iid"] = interactions_df["item_id"].astype("category")
interactions_df["iid"] = interactions_df["iid"].cat.codes

print(sorted(interactions_df.iid.unique())[:5])
print(sorted(interactions_df.uid.unique())[:5])
interactions_df.head()

In [ ]:
interactions_vec = np.zeros((interactions_df.uid.nunique(),
                             interactions_df.iid.nunique()))

for user_id, item_id in zip(interactions_df.uid, interactions_df.iid):
    interactions_vec[user_id, item_id] += 1


res = interactions_vec.sum(axis=1)
for i in range(len(interactions_vec)):
    interactions_vec[i] /= res[i]

In [ ]:
print(interactions_df.item_id.nunique())
print(items_ohe_df.item_id.nunique())
print(interactions_df.user_id.nunique())
print(users_ohe_df.user_id.nunique())

print(set(items_ohe_df.item_id.unique()) - set(interactions_df.item_id.unique()))

In [ ]:
iid_to_item_id = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()["item_id"]
item_id_to_iid = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]

uid_to_user_id = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
user_id_to_uid = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]

In [ ]:
items_ohe_df["iid"] = items_ohe_df["item_id"].apply(lambda x: item_id_to_iid[x])
items_ohe_df = items_ohe_df.set_index("iid")

users_ohe_df["uid"] = users_ohe_df["user_id"].apply(lambda x: user_id_to_uid[x])
users_ohe_df = users_ohe_df.set_index("uid")

In [ ]:
def triplet_loss(y_true, y_pred, n_dims=128, alpha=0.4):
    anchor = y_pred[:, 0:n_dims]
    positive = y_pred[:, n_dims:n_dims*2]
    negative = y_pred[:, n_dims*2:n_dims*3]
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    neg_dist = K.sum(K.square(anchor - negative), axis=1)

    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0)

    return loss

In [ ]:
def generator(items, users, interactions, batch_size=1024):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        for _ in range(batch_size):
            uid_i = randint(0, interactions.shape[0]-1)
            pos_i = np.random.choice(range(interactions.shape[1]), p=interactions[uid_i])
            neg_i = np.random.choice(range(interactions.shape[1]))
            uid_meta.append(users.iloc[uid_i])
            uid_interaction.append(interactions_vec[uid_i])
            pos.append(items.iloc[pos_i])
            neg.append(items.iloc[neg_i])
        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [np.array(uid_meta), np.array(uid_interaction)]


In [ ]:
def weighted_generator(items, users, interactions, batch_size=1024):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        for _ in range(batch_size):
            uid_i = np.random.choice(range(interactions.shape[0]), p=np.sum(interactions, axis=1) / np.sum(interactions))
            pos_i = np.random.choice(range(interactions.shape[1]), p=interactions[uid_i] / np.sum(interactions[uid_i]))
            neg_i = np.random.choice(range(interactions.shape[1]), p=(1 - interactions[uid_i]) / np.sum(1 - interactions[uid_i]))
            uid_meta.append(users.iloc[uid_i])
            uid_interaction.append(interactions_vec[uid_i])
            pos.append(items.iloc[pos_i])
            neg.append(items.iloc[neg_i])

        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [np.array(uid_meta), np.array(uid_interaction)]


In [ ]:
gen = weighted_generator(items=items_ohe_df.drop(["item_id"], axis=1),
                users=users_ohe_df.drop(["user_id"], axis=1),
                interactions=interactions_vec)

ret = next(gen)

print(f"вектор фичей юзера: {ret[0][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[0][1].shape}")
print(f"вектор 'хорошего' айтема: {ret[0][2].shape}")
print(f"вектор 'плохого' айтема: {ret[0][3].shape}")
print()
print(f"вектор фичей юзера: {ret[1][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[1][1].shape}")

In [ ]:
N_FACTORS = 128

ITEM_MODEL_SHAPE = (items_ohe_df.drop(["item_id"], axis=1).shape[1], )
USER_META_MODEL_SHAPE = (users_ohe_df.drop(["user_id"], axis=1).shape[1], )

USER_INTERACTION_MODEL_SHAPE = (interactions_vec.shape[1], )

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

In [ ]:
def item_model(n_factors=N_FACTORS):
    inp = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
    layer_1 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                               kernel_regularizer=keras.regularizers.l2(1e-6),
                               activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp)
    layer_2 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1)

    add = keras.layers.Add()([layer_1, layer_2])
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(add)

    return keras.models.Model(inp, out)


def user_model(n_factors=N_FACTORS):
    inp_meta = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
    inp_interaction = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)
    layer_1_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_meta)
    layer_1_interaction = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_interaction)
    layer_2_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1_meta)

    add = keras.layers.Add()([layer_1_meta, layer_2_meta])

    concat_meta_interaction = keras.layers.Concatenate()([add, layer_1_interaction])

    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(concat_meta_interaction)

    return keras.models.Model([inp_meta, inp_interaction], out)

i2v = item_model()
u2v = user_model()

ancor_meta_in = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
ancor_interaction_in = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

pos_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
neg_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

ancor = u2v([ancor_meta_in, ancor_interaction_in])
pos = i2v(pos_in)
neg = i2v(neg_in)

res = keras.layers.Concatenate(name="concat_ancor_pos_neg")([ancor, pos, neg])

model = keras.models.Model([ancor_meta_in, ancor_interaction_in, pos_in, neg_in], res)

In [ ]:
model_name = 'recsys_resnet_linear'

t_board = keras.callbacks.TensorBoard(log_dir=f'runs/{model_name}')

decay = keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, factor=0.8, verbose=1)

check = keras.callbacks.ModelCheckpoint(filepath=model_name + '/epoch{epoch}-{loss:.2f}.h5', monitor="loss")


In [ ]:
opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss=triplet_loss, optimizer=opt)

In [ ]:

item_model().summary()

In [ ]:
user_model().summary()

In [ ]:
model.summary()

In [ ]:
model.fit(generator(items=items_ohe_df.drop(["item_id"], axis=1),
                    users=users_ohe_df.drop(["user_id"], axis=1),
                    interactions=interactions_vec,
                    batch_size=64),
          steps_per_epoch=100,
          epochs=30,
          initial_epoch=0,
          callbacks=[decay, t_board, check]
)

In [ ]:
rand_uid = np.random.choice(list(users_ohe_df.index))

user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[rand_uid]
user_interaction_vec = interactions_vec[rand_uid]

rand_iid = np.random.choice(list(items_ohe_df.index))
item_feats = items_ohe_df.drop(["item_id"], axis=1).iloc[rand_iid]

user_vec = u2v.predict([np.array(user_meta_feats).reshape(1, -1),
                        np.array(user_interaction_vec).reshape(1, -1)])

item_vec = i2v.predict(np.array(item_feats).reshape(1, -1))

from sklearn.metrics.pairwise import euclidean_distances as ED

ED(user_vec, item_vec)

In [ ]:
users_meta_feats = users_ohe_df.drop(["user_id"], axis=1)
users_interaction_vec = interactions_vec
print(users_meta_feats.shape)
print(users_interaction_vec.shape)

np.array(users_meta_feats).shape

In [ ]:
users_vec = u2v.predict([np.array(users_meta_feats),
                        np.array(users_interaction_vec)])
print(users_vec.shape)
print(items_vecs.shape)

In [ ]:
dists = ED(users_vec, items_vecs)

print(dists.shape)

In [ ]:
top10_iids = np.argsort(dists, axis=1)[:,:10]

print(top10_iids.reshape(dists.shape[0], 10).shape)
print(top10_iids.shape)

top10_iids

In [ ]:
top10_iids_item = [iid_to_item_id[iid] for iid in top10_iids.reshape(-1)]
top10_iids_item = np.array(top10_iids_item).reshape(top10_iids.shape)
print(top10_iids_item.shape)

In [ ]:
df_dssm = pd.DataFrame(columns = ['user_id', 'item_id'])
df_dssm = pd.DataFrame({'user_id': [uid_to_user_id[uid] for uid in np.arange(top10_iids_item.shape[0])]})
df_dssm['item_id'] = list(top10_iids_item)
df_dssm = df_dssm.explode('item_id')
df_dssm['rank'] = df_dssm.groupby('user_id').cumcount() + 1
df_dssm = df_dssm.groupby('user_id').agg({'item_id': list}).reset_index()

df_dssm

In [ ]:
df_dssm.to_csv('dssm_recos.csv')